In [1]:
import os
import time
import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from utils.data import SleepDataLoader
from models import HeartRateNetwork

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
sub_ids = [x.split('_')[0] for x in os.listdir('data')]
data_loader = SleepDataLoader(sub_ids)
train_loader = data_loader.train(batch_size=1200)
val_loader = data_loader.val(batch_size=1200)
test_loader = data_loader.test(batch_size=1200)

In [49]:
for x, y in train_loader:
    print(x.shape)
    mask = torch.any(x != 0, dim=1).nonzero().squeeze()
    print(mask.shape)
    print(torch.index_select(x, 0, mask).shape)
    break

torch.Size([1200, 256])
torch.Size([1101])
torch.Size([1101, 256])


In [ ]:
callbacks = [EarlyStopping(monitor='val_BinaryF1Score', patience=3, mode='max', verbose=True),
             ModelCheckpoint(monitor='val_BinaryF1Score', mode='max')]

run_name = 'MESA_' + time.strftime("%Y%m%d-%H%M%S")
model = HeartRateNetwork(lr=0.0001, weight_decay=0.25)

In [ ]:
logger = WandbLogger(project='sleep-ihr', name=run_name, log_model='all')
# logger.experiment.config.update(args)

trainer = Trainer(accelerator='gpu', devices=1, max_epochs=1, logger=logger, callbacks=callbacks)
trainer.fit(model, train_loader, val_loader)

In [25]:
import numpy as np

# Assuming you have the arrays `data` and `labels`
data = np.array([[1, 2, 3], [0, 0, 0], [4, 5, 6]])
labels = np.array([0, 1, 0])

mask = np.any(data == 0, axis=1).nonzero()

In [34]:
mask[0].item()

1

In [27]:
labels[~mask]

TypeError: bad operand type for unary ~: 'tuple'

In [ ]:
x = np.array([1, 2, 3])
np.asarray(x == 1).nonzero()

In [ ]:
import pandas as pd
import numpy as np

# Create a sample dataframe
data = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
df = pd.DataFrame(data)
print(df)
desired_length = 5

# Pad the dataframe with zeros
df = df.reindex(range(desired_length), fill_value=0)

print(df)

In [ ]:
import pandas as pd
import numpy as np

# Create a sample dataframe
data = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
df = pd.DataFrame(data)

desired_length = 5

if len(df) < desired_length:
    # Calculate the number of rows to pad
    num_rows_to_pad = desired_length - len(df)
    
    # Create a dataframe with zeros
    zeros_df = pd.DataFrame(0, index=range(num_rows_to_pad), columns=df.columns)
    
    # Append the zeros dataframe to the original dataframe
    df = df.append(zeros_df, ignore_index=True)
print(df)